# 原本的permalink長這樣


In [2]:
permalink = 'http://djirs.judicial.gov.tw/fjud/index_1_S.aspx?p=9HeWsc4QZCsdTtczF7l%2FzrsVuYl47JuoFjnSV4hOj%2B0%3D'



# 可以切三段
- http://djirs.judicial.gov.tw/fjud/
- index_1_S.aspx?
- p=9HeWsc4QZCsdTtczF7l%2FzrsVuYl47JuoFjnSV4hOj%2B0%3D

# 第二段改成下面這段，就可以抓到全文

- FJUDQRY03_1_S.aspx?


### 原因應該是因為全文是包在一個框架內，點檢視原始碼會沒有，點檢視「頁框原始碼」就可看到。兩者的差別只在中間那段

In [4]:
# 摘出前後part
tmp = permalink.split('index_1_S.aspx?')
tmp


['http://djirs.judicial.gov.tw/fjud/',
 'p=9HeWsc4QZCsdTtczF7l%2FzrsVuYl47JuoFjnSV4hOj%2B0%3D']

In [7]:
#裝回去
k = tmp[0]+'FJUDQRY03_1_S.aspx?'+tmp[1]
k 

'http://djirs.judicial.gov.tw/fjud/FJUDQRY03_1_S.aspx?p=9HeWsc4QZCsdTtczF7l%2FzrsVuYl47JuoFjnSV4hOj%2B0%3D'

In [23]:
from bs4 import BeautifulSoup as BP
import requests
import pandas as pd

In [17]:
def content(url):
    
    res = requests.get(url) #header也不加了，可以直接只有url
#     print(res)
    html = BP(res.text,'lxml')
#     print(html)
    
    #判決書全部內容     
    text = html.find('pre').text.replace('\u3000\u3000\u3000','').replace('\r\n    ','\n')
    print(text)

In [27]:
raw = content(k)

臺灣臺北地方法院交通事件裁定　　九十年度交聲字第七四六號
　　異議人即
　　受處分人　甲○○
右列受處分人因違反道路交通管理處罰條例案件，對於臺北市交通事件裁決所中華民
國八十九年八月一日所為之處分（原處分案號：北市裁三字第裁00－000000
號），聲明異議，本院裁定如左：
主      文
原處分撤銷。
甲○○機車駕駛人，酒精濃度過量駕駛汽車，處罰鍰新臺幣壹萬貳仟元，並吊扣其駕
駛執照陸個月。
理      由
一、按道路交通安全規則所稱之汽車，依該規則第二條第一項第一款之規定，包括機
器腳踏車在內，又汽車駕駛人飲酒後其吐氣所含酒精成份超過每公升０．二五毫
克以上者，不得駕車，道路交通安全規則第一百十四條第二款定有明文。而道路
交通管理處罰條例（九十年一月十七日總統修正公佈之道路交通管理處罰條例部
分條文，其施行日期依同條例第九十三條規定須由行政院以命令定之，而行政院
係於九十年四月三日以臺九十交字第０一九三一七號令發布修正後之該條例自九
十年六月一日起施行，故以下所述及之本條例，如無特別註明，乃係修正生效後
之新法，而於敘述上如係九十年一月十七日修正，同年六月一日生效施行前之該
條例，則會在該條例下加註「舊法」以資辨明，先予敘明）所稱之汽車，依該條
例第三條第八款規定，亦包括機器腳踏車在內，而汽車駕駛人，酒精濃度過量駕
駛汽車者，處新臺幣（下同）六千元以上一萬二千元以下罰鍰，並當場禁止其駕
駛及吊扣其駕駛執照六個月，道路交通管理處罰條例第三十五條第一項第一款（
舊法）定有明文。
二、本件受處分人即異議人甲○○於八十八年十二月二十日凌晨零時二分許，騎乘車
號ＡＶＹ－０９１重型機車行經臺北市○○○路○段，因疑似酒後駕駛機車，遇
臺北市政府警察局中正第二分局交通分隊員警王文興予以攔停實施酒測，發現其
吐氣所含酒精成份為每公升零點三三毫克，員警隨即以受處分人有違反前揭道路
交通管理處罰條例第三十五條第一項第一款（舊法）規定之行為掣單舉發，受處
分人簽收告發通知單後，經員警向受處分人告知應到案處所及應到案日期，受處
分人未依限自行到案，原處分乃依前揭道路交通管理處罰條例第三十五條第一項
第一款規定（舊法）、違反道路交通管理事件統一裁罰標準及處理細則第四十四
條第一項規定逕行裁處受處分人罰鍰一萬二千元並予吊扣其駕駛執照六個月。
三、訊據受處分人否認有任何酒後駕車，並於前述

In [25]:
#wash function 吃str 清理文字，用在初步取出字號、聲請人、被告、主文、理由、判決日期等欄位，return df
def wash(raw):
    text = raw
    pos = [0,1,2]
    for i,j in enumerate(text):
        if bool(re.search("\\n主\s*文", j)):
            pos.append(i+1)
        if bool(re.search("\\n事\s*實", j)):
            pos.append(i+1)
        if bool(re.search("中\s*華\s*民\s*國\s*", j)):
            pos.append(i)
    if len(pos)>=5:
        
        #data是一個list
        #data[0] 字號
        data = [text[pos[0]]]
        
        #data[1] 聲請人
        data.append(re.sub('\u3000|\s*','',text[pos[1]]))
        
        #data[2] 被告
        data.append(re.sub('被告\u3000','',text[pos[2]]))
        
        #data[3] 接起來，主文
        t = ''
        for i in range(pos[3],pos[4]):
            t = t+text[i]
            t = re.sub('\s*','',t)
        data.append(t)
        
        #data[4] 接起來，事實理由
        t = ''
        for i in range(pos[4],pos[5]):
            t = t+text[i]
            t = re.sub('\s*','',t)
        data.append(t)
        
        #data[5] 日期
        data.append(re.sub('\s*','',text[pos[5]]))
        
    #是不是有其他狀況，要NaN處理?
    
    #data[6] 全文   
    data.append(re.sub('\s*|\r|\n','',raw))
    
    #data[7] 永久連結   
    data.append(permalink) #add permalink
        
    df = pd.DataFrame.from_dict({'row1':data}, orient='index')
    df.columns = ['字號','聲請人','被告','主文','事實理由','判決日期','全文','永久連結'] #add permalink
    return df


#alcohol function 吃list 是從全文這欄中取出酒精濃度的數值，若沒取到也會吐回０．２５（因為有的判決沒寫實際數字．．．只寫超過零點二五），
#因此需注意搭配關鍵字搜尋結果，如酒精濃度、駕駛動力交通工具等篩選
def alcohol(column):
    alcohol = []
    for i in column:
        matches = re.findall('酒精濃度[^a-zA-Z0-9_]*每公升.\...毫克|酒精濃度[^a-zA-Z0-9_]*.\...mg/L',i)
        matches = [i for i in matches if '0.25' not in i]
        if len(matches)>0:
            a = float(re.sub('酒精濃度[^a-zA-Z0-9_]*每公升|酒精濃度達|酒精濃度[^a-zA-Z0-9_]*|毫克|mg/L','',matches[0]).replace('零','0').replace('一','1').replace('二','2').replace('三','3').replace('四','4').replace('五','5').replace('六','6').replace('七','7').replace('八','8').replace('九','9').replace('十','10').replace('點','.'))
            alcohol.append(a)
        else:
            alcohol.append(0.25)
    return alcohol

#recidivism function 吃list 從全文這欄中取出是否為累犯/初犯，但不見得會記述，因此多一個未提及選項
def recidivism(column):
    recidivism = []
    for i in df['全文']:
        recid = re.search('累犯',i)
        first = re.search('初犯',i)
        deferred = re.search('緩起訴',i)
        t = ''
        if bool(recid):
            t = t+'累犯'
        if bool(first):
            t = t+'初犯'
        if bool(deferred):
            t = t+'曾緩起訴'
        if t=='':
            t = '未提及'
        recidivism.append(t)
    return recidivism

#penalty function 吃list 從主文這欄取出刑罰，因為看到一個判決裡面有多人，因此先保留為list的形式，後續可再討論
def penalty(column):
    imprisonment = []
    fines = []
    commutation = []
    for i in column:
        im = re.findall('處有期徒刑[^a-zA-Z0-9_]*月',i)
        im = [re.sub('處有期徒刑|月','',i) for i in im]   
        imprisonment.append(im)
        fi = re.findall('罰金新臺幣[^a-zA-Z0-9_]*元',i)
        fi = [re.sub('罰金新臺幣|元.*$','',i) for i in fi]
        fines.append(fi)
        co = re.findall('如易科罰金',i)
        if len(co)>0:
            co = 'true'
        else:
            co = 'false'
        commutation.append(co)
    return imprisonment,fines,commutation

In [26]:
df = pd.DataFrame()
df.append(wash(raw))
df

TypeError: 'NoneType' object is not iterable